# Parametrización del Algoritmo de Lobo Gris Discreto (DGWO) para su aplicación en la Distribución de Instalaciones con Áreas Desiguales (UAFLP)

El proceso de *Parametrización* o tuneo de los parámetros del DGWO para su aplicación en problemas UAFLP consiste en experimentar diferentes valores de los parámetros del algoritmo (p.e., diferentes valores de tamaño de la manada) con el fin de conocer el conjunto de parámetros que mejor se adapta (es decir, que obtiene la mejor solución) para la solución de los problemas UAFLP, previo a su aplicación al caso de estudio real.

Para ello, el DGWO se aplicará a las siguientes instancias de datos recolectadas de la literatura relacionada con el UAFLP:

| Instancia | Num Dptos | Lados de Instalación ($W x H$) | Tasa de Aspecto | Norma de Distancia | Referencia              |
|-----------|-----------|--------------------------------|-----------------|--------------------|-------------------------|
| O7        |     7     |        ($8.54$ x $13.00$)      |   $\alpha = 4$  |     Rectilínea     | Meller et al. (1998)    |
| O8        |     8     |       ($11.31$ x $13.00$)      |   $\alpha = 4$  |     Rectilínea     | Meller et al. (1998)    |
| O9        |     9     |       ($12.00$ x $13.00$)      |   $\alpha = 4$  |     Rectilínea     | Meller et al. (1998)    |
| vC10R-a   |    10     |       ($25.00$ x $51.00$)      |   $\alpha = 5$  |     Rectilínea     | van Camp et al. (1992)  |
| Garment   |    11     |       ($25.00$ x $45.60$)      |   $\alpha = 4$  |     Rectilínea     | Zapata et al. (2023)    |
| MB12      |    12     |        ($6.00$ x $8.00$)       |   $\alpha = 4$  |     Rectilínea     | Bozer and Meller (1997) |

**Referencias**:

* Bozer, Y.A., Meller, R.D. (1997). A reexamination of the distance-based facility layout problem. IIE Trans., 29. pp. 549-560
* Meller, R.D., Narayanan, V., Vance, P.H. (1998). Optimal facility layout design. Oper. Res. Let., 23. pp. 117-127
* van Camp, D.J., Carter, M.W., Vannelli, A. (1992). A nonlinear optimization approach for solving facility layout problems. Eur. J. Oper. Res., 57. pp. 174-189 
* Zapata-Cortés, J.A., Arango-Serna, M.D., Cáceres-Gelvez, S. (2023). Genetic algorithm for the optimization of the Unequal-Area Facility Layout Problem. In: Handbook on Decision Making, 3. pp. 399-418

### Parámetros del DGWO

El algoritmo DGWO incluye los siguientes parámetros, los cuales se buscarán ajustar para la optimización del UAFLP:

* `tam_manada` ($M$): Hace referencia al tamaño de la manada, o el número de individuos (lobos) que comprenden la manada dentro del algoritmo.
* `n_lideres` ($N$): Hace referencia al número de lobos líderes, los cuales guiarán la búsqueda la presa (mejor solución). Solo puede tomar los valores `1` ($\alpha$) y `3` ($\alpha$, $\beta$ y $\delta$). Los demás lobos son tratados como lobos de la manada que seguirán a uno de los líderes en los comportamientos de búsqueda y ataque.
* `theta_1` ($\theta_1$): Hace referencia al porcentaje del tamaño del movimiento que debe realizarse durante el comportamiento de búsqueda de la presa. Es un parámetro que debe estar entre `0` y `1`.
* `theta_2` ($\theta_2$): Hace referencia al porcentaje de la distancia mínima que debe tener cada lobo en la manada con respecto a uno de los líderes. Es un parámetro que debe estar entre `0` y `1`.
* `theta_3` ($\theta_3$): Hace referencia a la probabilidad de seleccionar al lobo alfa ($\alpha$) entre los lobos líderes para reducir la distancia de los lobos en la manada. Es un parámetro que debe estar entre `0` y `1`, y debe ser igual a `1` si `n_lideres = 1`.

Las instancias de datos se extraen de archivos de datos a través de la función `obtener_datos` de la clase `ModeloUAFLP` y el algoritmo DGWO será utilizado, a través de la clase `AlgLoboGris`, para optimizar estas instancias usando los siguientes valores de los parámetros:

* `tam_manada` ($M$): `[6, 8, 10, 12, 15]`
* `n_lideres` ($N$): `[1, 3]`
* `theta_1` ($\theta_1$): `[0.2, 0.4, 0.6, 0.8]`
* `theta_2` ($\theta_2$): `[0.2, 0.4, 0.6, 0.8]`
* `theta_3` ($\theta_3$): `[0.2, 0.4, 0.6, 0.8]` o `[1]` si `n_lideres = 1`

## 1. Clase `ModeloUAFLP`

Clase para el modelo UAFLP:

In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

class ModeloUAFLP:
    
    # Constructor de la clase (argumentos requeridos para crear la clase)
    def __init__(self, n_dptos:int=None, areas_dptos:np.ndarray=None, flujo_materiales:np.ndarray=None,
                lados_instalacion:np.ndarray=None, tasa_aspecto_max:int=None, costo_manejo_unit:float=None,
                nombres_dptos:list=None, mejor_valor:float=None, archivo_datos:str=None) -> None:
        
        if archivo_datos == None:
            self.n_dptos = n_dptos
            self.departamentos = np.arange(1, self.n_dptos + 1)
            self.areas_dptos = areas_dptos
            self.flujo_materiales = flujo_materiales
            self.lados_instalacion = lados_instalacion # ancho, largo
            self.tasa_aspecto_max = tasa_aspecto_max
            self.nombres_dptos = nombres_dptos
            self.mejor_val = mejor_valor
            self.costo_manejo_unit = costo_manejo_unit
            
            # Definir matriz de costo de manejo unitario
            if self.costo_manejo_unit == None:
                self.matriz_costos_dptos = np.ones((self.n_dptos, self.n_dptos))
            else:
                self.matriz_costos_dptos = np.full((self.n_dptos, self.n_dptos), self.costo_manejo_unit)
        else: # Extraer datos de archivo de datos
            self.obtener_datos(arch_datos=archivo_datos)

    # Método para leer archivo de datos
    def obtener_datos(self, arch_datos:str):

        # Cambiar el directorio
        path = os.path.dirname(os.path.realpath(__file__)) + '\\uaflp-instances'

        try:
            os.chdir(path)
        except FileNotFoundError:
            print(f'Directory: {path} does not exist')
            sys.exit()
        except NotADirectoryError:
            print(f'{path} is not a directory')
            sys.exit()

        # Abrir el archivo de datos
        with open(arch_datos) as f:
            lines = f.readlines()

        # Leer cada linea del archivo
        pr_linea = lines[0].split() # [n_dptos, ancho_inst, largo_inst, tasa_asp, costo_unit, mejor_val]
        seg_linea = lines[1].split() # [areas_dptos 1,..., n]
        ter_linea = lines[2].split() # [nombres_dptos 1,..., n] o ['None']
        fl_mats = [] # lista de listas flujos de materials
        for l in lines[3:]:
            ln = [float(x) for x in l.split()]
            fl_mats.append(ln)

        # Guardar los parametros del modelo UAFLP
        self.n_dptos = int(pr_linea[0])
        self.departamentos = np.arange(1, self.n_dptos + 1)
        self.lados_instalacion = np.array([float(x) for x in pr_linea[1:3]])
        self.tasa_aspecto_max = int(pr_linea[3])
        self.costo_manejo_unit = float(pr_linea[4])
        self.matriz_costos_dptos = np.full((self.n_dptos, self.n_dptos), self.costo_manejo_unit)
        self.mejor_val = float(pr_linea[-1])
        self.areas_dptos = np.array([float(x) for x in seg_linea])

        if len(ter_linea) > 1:
            self.nombres_dptos = [x for x in ter_linea]
        else:
            self.nombres_dptos = None

        self.flujo_materiales = np.array(fl_mats)
        

    # Método para decodificar la solución 
    def decodificar_solucion(self, solucion:np.ndarray):
        self.identificar_bahias(solucion)
        self.calcular_lados_centros()
    
    # Identificar bahías
    def identificar_bahias(self, solucion):
        self.bahias = []
        dpts_bahias = []

        for ind, bah in enumerate(solucion[1]):
            dpto = solucion[0, ind] # Identificar dpto en posicion

            if ind == 0 or bah == 0:
                dpts_bahias.append(dpto) # Añadir dptos en bahía
                if bah == 1:
                    self.bahias.append(dpts_bahias) # Añadir bahía a bahías
                    dpts_bahias = []
            elif bah == 1:
                dpts_bahias.append(dpto) # Añadir dptos en bahía
                self.bahias.append(dpts_bahias) # Añadir bahía a bahías
                dpts_bahias = []
    
    # Calcular dimensiones de lados y centroides de los departamentos
    def calcular_lados_centros(self):
        self.centroides_dptos = [0] * self.n_dptos
        self.lados_dptos = [0] * self.n_dptos

        contador_ancho = 0
        for bah in self.bahias:

            area_bahia = 0
            for dpto in bah:
                area_bahia += self.areas_dptos[dpto-1] # area bahia = suma(areas dptos en bahia)

            ancho_bahia = area_bahia / self.lados_instalacion[1] # ancho bahia = area de bahia / largo de instalacion

            contador_largo = 0
            for dpto in bah:
                largo_dpto = self.areas_dptos[dpto-1] / ancho_bahia # largo dpto = area dpto / ancho bahia
                self.lados_dptos[dpto-1] = [ancho_bahia, largo_dpto] # Nota: ancho bahia == ancho dpto en bahia
                centro_x = contador_ancho + ancho_bahia / 2
                centro_y = contador_largo + largo_dpto / 2
                self.centroides_dptos[dpto-1] = [centro_x, centro_y]
                contador_largo += largo_dpto

            contador_ancho += ancho_bahia

    # Dibujar layout de planta de la solucion
    def dibujar_layout(self, solucion:np.ndarray):
        if self.nombres_dptos == None:
            self.nombres_dptos = [f'Dpto {d}' for d in self.departamentos]

        self.decodificar_solucion(solucion)

        if self.lados_instalacion[0] > self.lados_instalacion[1]:
            fig = plt.figure(dpi=300, figsize=(6, 4))
        else:
            fig = plt.figure(dpi=300, figsize=(4, 6))

        plt.rcParams.update({'font.size': 6})
        ax = fig.add_subplot(111)
        ax.set_xlim([0, self.lados_instalacion[0]])
        ax.set_ylim([0, self.lados_instalacion[1]])
        ax.set_xticks([0, self.lados_instalacion[0]])
        ax.set_yticks([0, self.lados_instalacion[1]])

        esq_x = 0
        for bah in self.bahias:

            esq_y = 0
            for dpt in bah:
                rect = plt.Rectangle((esq_x, esq_y), width=self.lados_dptos[dpt-1][0],
                                     height=self.lados_dptos[dpt-1][1], facecolor='white',
                                     edgecolor='black')
                plt.text(self.centroides_dptos[dpt-1][0], self.centroides_dptos[dpt-1][1],
                         f'{self.nombres_dptos[dpt-1]}', horizontalalignment='center',
                         verticalalignment='top')
                plt.plot(self.centroides_dptos[dpt-1][0], self.centroides_dptos[dpt-1][1],
                         color='black', marker=None, markersize=2)
                ax.add_patch(rect)

                esq_y += self.lados_dptos[dpt-1][1]

            esq_x += self.lados_dptos[dpt-1][0]

        plt.show()

    # Calcular la función fitness
    def calcular_fitness(self, solucion:np.ndarray):
        self.decodificar_solucion(solucion)
        self.calcular_distancias()
        self.calcular_mhc()

        k = 3

        contador_no_factibles = 0
        for i in solucion[0]:
            tasa_aspecto_dpto = max(self.lados_dptos[i-1][0], self.lados_dptos[i-1][1]) / min(self.lados_dptos[i-1][0], self.lados_dptos[i-1][1])

            if tasa_aspecto_dpto > self.tasa_aspecto_max:
                contador_no_factibles += 1

        fitness = self.mhc + self.mhc * (contador_no_factibles ** k)

        return round(fitness, ndigits=2)

    # Calcular las distancias rectilíneas entre departamentos
    def calcular_distancias(self):
        self.distancias_dptos = np.zeros((self.n_dptos, self.n_dptos))

        for i in self.departamentos:
            for j in self.departamentos[i:]:
                dist_rectil_x = np.abs(self.centroides_dptos[i-1][0] - self.centroides_dptos[j-1][0])
                dist_rectil_y = np.abs(self.centroides_dptos[i-1][1] - self.centroides_dptos[j-1][1])

                self.distancias_dptos[j-1, i-1] = dist_rectil_x + dist_rectil_y

    # Calcular el costo total de manejo de materiales entre departamentos
    def calcular_mhc(self):
        
        self.mhc = np.sum(self.matriz_costos_dptos * self.distancias_dptos * self.flujo_materiales)

## 2. Clase `AlgLoboGris`

Clase para el algoritmo DGWO, heredando los métodos de la clase del modelo UAFLP:

In [2]:
import numpy as np
from copy import deepcopy
from time import time
from uaflp import ModeloUAFLP

class AlgLoboGris(ModeloUAFLP):

    # Constructor de la clase (hereda las caracteristicas de la clase ModeloUAFLP)
    def __init__(self, n_dptos: int = None, areas_dptos: np.ndarray = None, flujo_materiales: np.ndarray = None, lados_instalacion: np.ndarray = None, tasa_aspecto_max: int = None, costo_manejo_unit: float = None, nombres_dptos: list = None, mejor_valor: float = None, archivo_datos: str = None) -> None:
        super().__init__(n_dptos, areas_dptos, flujo_materiales, lados_instalacion, tasa_aspecto_max, costo_manejo_unit, nombres_dptos, mejor_valor, archivo_datos)

    # Correr algoritmo de lobo gris
    def optimizacion_lobo_gris(self, tam_manada:int, n_lideres:int, theta_1:float, theta_2:float, theta_3:float=1, tiempo_lim:int=600):
        """_summary_

        Args:
            tam_manada (int): Hace referencia al tamaño de la manada, o el número de lobos (soluciones) 
                            que comprenden la manada dentro del algoritmo.
            n_lideres (int): Hace referencia al número de lobos líderes, los cuales guiarán la búsqueda 
                            la presa (mejor solución). Solo puede tomar los valores `1` (lobo alfa) o `3` 
                            (lobos alfa, beta y delta). Los demás lobos son tratados como lobos de la 
                            manada (lobos omega).
            theta_1 (float): Hace referencia al porcentaje del tamaño del movimiento (n * theta_1) que debe 
                            realizarse durante el comportamiento de búsqueda de la presa. Es un parámetro 
                            que debe estar entre 0 y 1.
            theta_2 (float): Hace referencia al porcentaje de la distancia mínima (n * theta_2) que debe 
                            tener cada lobo en la manada con respecto a uno de los líderes. Es un parámetro 
                            que debe estar entre 0 y 1.
            theta_3 (float): (Opcional si n_lideres = 1) Hace referencia a la probabilidad de seleccionar al 
                            lobo alfa (mejor solución) entre los lobos líderes para reducir la distancia de 
                            los lobos en la manada. Es un parámetro que debe estar entre 0 y 1.
            tiempo_lim (int, opcional): Tiempo de corrida del algoritmo en segundos. Por defecto 600.

        Raises:
            ValueError: Si el valor del parámetro tam_manada no está entre n_lideres + 1 y 15 (inclusive).
            ValueError: Si el valor del parámetro n_lideres no es 1 o 3.
            ValueError: Si el valor de los parámetros theta_1, theta_2 y theta_3 no están entre 0 y 1.

        Returns:
            dict: Diccionario con las mejores soluciones ('sols_uaflp') y sus respectivas funciones fitness
                    ('vals_fitness')
        """
        if tam_manada not in range(n_lideres + 1, 16):
            raise ValueError(f'El valor del parametro tam_manada deberia ser entre n_lideres + 1 y 15 (inclusive): {tam_manada}')
        
        if n_lideres not in {1, 3}:
            raise ValueError(f'El valor del parametro n_lideres deberia ser igual a 1 o 3: {n_lideres}')

        if theta_1 <= 0 or theta_1 >= 1 or theta_2 <= 0 or theta_2 >= 1 or theta_3 <= 0 or theta_3 > 1:
            raise ValueError(f'El valor de los parametros theta_1, theta_2 and theta_3 deberia estar entre 0 y 1: {[theta_1, theta_2, theta_3]}')

        if n_lideres == 1:
            theta_3 = 1
            print('El valor del parametro theta_3 se ha igualado a 1 ya que el parametro n_lideres es igual a 1')

        timeout = time() + tiempo_lim
        lobos_sols = []
        fit_sols = []

        self.generar_manada(tam_manada)
        self.identificar_lideres(n_lideres)
        while time() < timeout:
            
            self.buscar_presa(theta_1)
            self.atacar_presa(n_lideres, theta_2, theta_3)

            lobos_sols.append(self.lobos_lideres[0])
            fit_sols.append(self.fit_lideres[0])

        return {'sols_uaflp': lobos_sols, 'vals_fitness': fit_sols}

    # Inicializar poblacion de la manada
    def generar_manada(self, tam_manada):

        self.manada = [0] * tam_manada
        self.fit_manada = [0] * tam_manada

        for lobo_ind in range(tam_manada):
            lobo_dpts = list(np.random.permutation(self.departamentos))
            lobo_bahs = list(np.random.randint(0, 2, self.n_dptos - 1)) + [1]
            self.manada[lobo_ind] = np.array([lobo_dpts, lobo_bahs])
            self.fit_manada[lobo_ind] = self.calcular_fitness(self.manada[lobo_ind])

    # Identificar lobos lideres ([alpha]) si n_lideres = 1, ([alpha, beta, delta]) si n_lideres = 3
    def identificar_lideres(self, n_lideres):

        self.lobos_lideres = [0] * n_lideres
        self.fit_lideres = [0] * n_lideres

        for ind in range(n_lideres):
            p_min = np.argmin(self.fit_manada)
            self.lobos_lideres[ind] = self.manada[p_min]
            self.fit_lideres[ind] = self.fit_manada[p_min]
            self.manada.pop(p_min)
            self.fit_manada.pop(p_min)

    # Ejecutar comportamiento de busqueda de la presa
    def buscar_presa(self, theta_1):
        
        tam_movimiento = np.floor(self.n_dptos * theta_1).astype(int)
        
        for ind, lobo in enumerate(self.manada):

            lobo_dp = deepcopy(lobo[0])
            lobo_bh = deepcopy(lobo[1])

            # Realizar movimiento en elementos de lobos
            lobo_dp = self.realizar_movimiento(lobo_dp, tam_movimiento)
            lobo_bh = self.realizar_movimiento(lobo_bh, tam_movimiento)

            if lobo_bh[-1] != 1:
                lobo_bh[-1] = 1

            # Calcular funcion fitness del nuevo lobo
            nuevo_lobo = np.array([list(lobo_dp), list(lobo_bh)])
            fit_nuevo_lobo = self.calcular_fitness(nuevo_lobo)

            # Comparar y reemplazar (si aplica) nuevo lobo generado
            if fit_nuevo_lobo < self.fit_lideres[0]:

                self.insertar_nuevo_lider(nuevo_lobo, fit_nuevo_lobo, ind)
                break # Romper busqueda para iniciar ataque

            elif fit_nuevo_lobo < self.fit_manada[ind]:

                # Reemplazar nuevo lobo en manada
                self.manada[ind] = nuevo_lobo
                self.fit_manada[ind] = fit_nuevo_lobo

    # Ejecutar comportamiento de ataque a la presa
    def atacar_presa(self, n_lideres, theta_2, theta_3=None):
        
        da = np.floor(self.n_dptos * theta_2)

        for ind_md, lobo in enumerate(self.manada):

            nuevo_lobo = deepcopy(lobo)
            lider = None
            if n_lideres == 1:
                lider = self.lobos_lideres[0]
            else:
                rand = np.random.random()
                if rand < theta_3:
                    lider = self.lobos_lideres[0]
                elif rand < (theta_3 + (1 - theta_3)/2):
                    lider = self.lobos_lideres[1]
                else:
                    lider = self.lobos_lideres[2]

            # Calcular y reducir distancia con el lider para elemento con dptos
            dist_dptos = self.calcular_dist_lider(nuevo_lobo[0], lider[0])

            ind_dp = 0
            while dist_dptos > da:

                nuevo_lobo[0] = self.reducir_dist_dptos(nuevo_lobo[0], lider[0], ind_dp)
                dist_dptos = self.calcular_dist_lider(nuevo_lobo[0], lider[0])
                ind_dp += 1

            # Calcular y reducir distancia con el lider para elemento de bahias
            dist_bahias = self.calcular_dist_lider(nuevo_lobo[1], lider[1])

            ind_bh = 0
            while dist_bahias > da:

                nuevo_lobo[1] = self.reducir_dist_bahias(nuevo_lobo[1], lider[1], ind_bh)
                dist_bahias = self.calcular_dist_lider(nuevo_lobo[1], lider[1])
                ind_bh += 1

            fit_nuevo_lobo = self.calcular_fitness(nuevo_lobo)

            # Comparar y reemplazar (si aplica) nuevo lobo generado
            if fit_nuevo_lobo < self.fit_lideres[0]:

                 self.insertar_nuevo_lider(nuevo_lobo, fit_nuevo_lobo, ind_md)

            elif fit_nuevo_lobo < self.fit_manada[ind_md]:

                # Reemplazar nuevo lobo en manada
                self.manada[ind_md] = nuevo_lobo
                self.fit_manada[ind_md] = fit_nuevo_lobo

    # Metodo para realizar movimiento en elemento de lobos
    def realizar_movimiento(self, elem_lobo, tam_movimiento):

        pos_inicio, pos_fin = np.random.randint(0, self.n_dptos - tam_movimiento + 1, 2)
        pos_movimiento = np.arange(pos_inicio, pos_inicio + tam_movimiento)
        movimiento = elem_lobo[pos_movimiento]
        elem_lobo = np.delete(elem_lobo, pos_movimiento)
        elem_lobo = np.insert(elem_lobo, pos_fin, movimiento)

        return elem_lobo
    
    # Metodo para reemplazar nuevo lider en lideres y desplazar lobo delta a manada
    def insertar_nuevo_lider(self, nuevo_lobo, fit_nuevo_lobo, ind_manada):
        
        # Agregar nuevo alpha a los lideres
        self.lobos_lideres.insert(0, nuevo_lobo)
        self.fit_lideres.insert(0, fit_nuevo_lobo)

        # Desplazar antiguo lobo delta a la manada
        self.manada[ind_manada] = self.lobos_lideres[-1]
        self.fit_manada[ind_manada] = self.fit_lideres[-1]

        # Eliminar antiguo lobo delta de los lideres
        self.lobos_lideres.pop()
        self.fit_lideres.pop()

    # Metodo para calcular distancia entre lobo en manada y lobo lider
    def calcular_dist_lider(self, elem_lobo, elem_lider):

        distancia = 0
        for ind in range(len(elem_lider)):
            if elem_lobo[ind] != elem_lider[ind]:
                distancia += 1

        return distancia
    
    # Metodo para reducir la distancia entre el lobo en manada y lobo lider (para elemento con dptos)
    def reducir_dist_dptos(self, elem_lobo, elem_lider, ind_actual):

        dpto_lobo = elem_lobo[ind_actual]
        dpto_lider = elem_lider[ind_actual]
        if dpto_lobo != dpto_lider:
            pos_cambio = np.where(elem_lobo == dpto_lider)[0]
            elem_lobo[ind_actual] = dpto_lider
            elem_lobo[pos_cambio] = dpto_lobo

        return elem_lobo
    
    # Metodo para reducir la distancia entre el lobo en manada y lobo lider (para elemento con bahias)
    def reducir_dist_bahias(self, elem_lobo, elem_lider, ind_actual):

        if elem_lobo[ind_actual] != elem_lider[ind_actual]:
            elem_lobo[ind_actual] = elem_lider[ind_actual]

        return elem_lobo

## 3. Celda principal para parametrización del algoritmo DGWO

### Valores de los parámetros a utilizar:

* `tam_manada` ($M$): `[6, 8, 10, 12, 15]`
* `n_lideres` ($N$): `[1, 3]`
* `theta_1` ($\theta_1$): `[0.2, 0.4, 0.6, 0.8]`
* `theta_2` ($\theta_2$): `[0.2, 0.4, 0.6, 0.8]`
* `theta_3` ($\theta_3$): `[0.2, 0.4, 0.6, 0.8]` o `[1]` si `n_lideres = 1`

In [ ]:
# Valores de los parámetros

vals_manada = [10] # [6, 8, 10, 12, 15]
vals_lideres = [3] # [1, 3]
vals_theta_1 = [0.4] # [0.2, 0.4, 0.6, 0.8]
vals_theta_2 = [0.4] # [0.2, 0.4, 0.6, 0.8]
vals_theta_3 = [0.2, 0.4, 0.6, 0.8] # [0.2, 0.4, 0.6, 0.8]

In [ ]:
import pandas as pd

# Identificar el archivo y leer las instancias de datos
with open('./uaflp-instances/uaflp_instances.txt') as f:
    insts = f.readlines()

# Crear diccionario para guardar los resultados
resultados = {'inst': [],
              'n_dptos': [],
              'mejor_val': []}

# Guardar los datos principales del problema
for ln in insts:

    inst = ln.split()[0]

    prob = ModeloUAFLP(archivo_datos=inst)

    resultados['inst'].append(inst)
    resultados['n_dptos'].append(prob.n_dptos)
    resultados['mejor_val'].append(prob.mejor_val)

# Correr los diferentes valores de los parámetros
for val_mnd in vals_manada:
    for val_ld in vals_lideres:
        for val_t1 in vals_theta_1:
            for val_t2 in vals_theta_2:
                for val_t3 in vals_theta_3:

                    n_rondas = 0
                    while n_rondas < 10:
                    
                        resultados[f'set_{val_t3}_{n_rondas}_UB'] = []
                        resultados[f'set_{val_t3}_{n_rondas}_gap'] = []
                        resultados[f'set_{val_t3}_{n_rondas}_sol'] = []
                    
                        for inst in resultados['inst']:

                            dgwo = AlgLoboGris(archivo_datos=inst)

                            res_dgwo = dgwo.optimizacion_lobo_gris(tam_manada=val_mnd, n_lideres=val_ld, theta_1=val_t1,
                                                                theta_2=val_t2, theta_3=val_t3, tiempo_lim=300)
                            
                            uaflp_ub = res_dgwo['vals_fitness'][-1]
                            gap = (uaflp_ub - dgwo.mejor_val)/dgwo.mejor_val
                            uaflp_sol = res_dgwo['sols_uaflp'][-1]

                            resultados[f'set_{val_t3}_{n_rondas}_UB'].append(uaflp_ub)
                            resultados[f'set_{val_t3}_{n_rondas}_gap'].append(gap)
                            resultados[f'set_{val_t3}_{n_rondas}_sol'].append(uaflp_sol)

                            print(f'Terminando set de parametros {val_t3} con {n_rondas} rondas para la inst {inst} con UB = {uaflp_ub}')

                        # if val_ld == 1:
                        #     break
                        
                        n_rondas += 1

df = pd.DataFrame(resultados)
df.set_index('inst', inplace=True)

df.to_excel('parametrizacion_dgwo_theta3.xlsx')

In [3]:
lst_parametros = ['tam_manada', 'n_lideres', 'theta_1', 'theta_2', 'theta_3']
lst_vals_params = [[6, 8, 10, 12, 15], [1, 3], [0.2, 0.4, 0.6, 0.8], [0.2, 0.4, 0.6, 0.8], [0.2, 0.4, 0.6, 0.8]]

for param, vals_param in zip(lst_parametros, lst_vals_params):
    print(param, vals_param)

tam_manada [6, 8, 10, 12, 15]
n_lideres [1, 3]
theta_1 [0.2, 0.4, 0.6, 0.8]
theta_2 [0.2, 0.4, 0.6, 0.8]
theta_3 [0.2, 0.4, 0.6, 0.8]
